In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Set Global Variables and Create Dataframes

Dataframe Name | Table Name
| --- | --- |
df_epp | Expenditures Per Pupil
df_nxp | Inexperienced Teachers and Principals
df_ocert | Teachers Teaching Out of Certification
df_y | Annual Regents Exams

In [ ]:
df_epp = pd.read_excel("../Data/SRC2022/Expenditures Per Pupil.xlsx")
df_nxp = pd.read_excel("../Data/SRC2022/Inexperienced Teachers and Principals.xlsx")
df_ocert = pd.read_excel("../Data/SRC2022/Teachers Teaching Out of Certification.xlsx")
df_y = pd.read_excel("../Data/SRC2022/Annual Regents Exams.xlsx")

Variable Name | Meaning
| --- | --- |
chosen_test_subjects | The Annual Regents Exams we plan to use to measure school performance

In [18]:
chosen_test_subjects = ('Regents Common Core Algebra I', 'Regents Common Core English Language Art', 'Regents Phy Set/Earth Sci')

### Trimming Dataframes
#### Defining a Trimming Function that will only keep the columns we wish and only keep the rows that have our desired YEAR and SUBGROUP_NAME of "All Students"

In [33]:
def trimmer(df: pd.DataFrame, *column_names, year: int=2021, year_col_name: str="YEAR")->pd.DataFrame:
    """
    Keeps only the columns specified, only the rows whose year agrees with the year argument, 
    and if there is a SUBGROUP column then it keeps only rows with the subgroup classification 
    of "All Students". 
    This is NOT an in-place method. 
    """
    # Trimming Rows
    if year_col_name in df.columns:
        df = df.loc[df[year_col_name]==year]
    if "SUBGROUP_NAME" in df.columns:
        df = df.loc[df["SUBGROUP_NAME"]=="All Students"] #<-----This drops all rows that do not have "All Students" as the "SUBGROUP" column entry. 
    if "SUBJECT" in df.columns:
        df = df.loc[df.SUBJECT.isin(chosen_test_subjects)]
    # Trimming Columns
    return df[list(column_names)]

Note that we no longer have to look for ENTITY_CD ENTRIES that don't end in 0000.

#### Applying the Trimming Function

In [44]:
# Expenditures Per Pupil: Keep only rows with the year 2021 and keeping only school ID, Name and Normalized Expenditures
df_epp_t = trimmer(df_epp,'ENTITY_NAME', 'ENTITY_CD', 'PER_FED_STATE_LOCAL_EXP')
df_nxp_t = trimmer(df_nxp,'ENTITY_CD', 'PER_TEACH_INEXP')
df_ocert_t = trimmer(df_ocert,'ENTITY_CD', "PER_OUT_CERT")

# df_y: Drop NaN values and make INSTITUTION_ID a string
df_y_c = df_y.dropna(subset=['INSTITUTION_ID'], ignore_index=True)

# df_y: Trim and Pivot
df_y_t = trimmer(df_y_c, 'INSTITUTION_ID', 'ENTITY_CD', 'ENTITY_NAME', 'SUBJECT', "PER_PROF")


We had a lot of trouble with pivoting before. Now, hopefully the code all works well and we can just pivot. 
#### Checking for duplicates. 

In [35]:
df_y_t.ENTITY_CD.value_counts()

ENTITY_CD
211103040000    3
261600860985    3
220909040010    3
211701040001    3
211103040001    3
               ..
310200010177    1
310200010167    1
310200010131    1
310200010126    1
671002040001    1
Name: count, Length: 429, dtype: int64

Success! We have at most 3 instances for each ENTITY_CD, one for each type of exam we decided on. 

But if we look on the row of the last cell, it says "Length: <font color='red'><del>218</del></font> 429". So there are only <font color='red'><del>218</del></font> 429 values. That means there are only <font color='red'><del>218</del></font> 429 schools in the list now. We get the same number when using the INSTITUTION_ID column, so it's not the choice of id:

In [36]:
len(df_y_t.INSTITUTION_ID.unique())

429

But if we go back to the original Regents Exams Dataframe, we get almost 3,000 schools. Does this mean that 90% of these didn't have an exam entry for our matching tests? 

In [16]:
df_y.ENTITY_CD.value_counts()

ENTITY_CD
60601040000     214
60601040003     212
62901040000     195
62901040002     195
10100860907     193
               ... 
320900010323      1
320900010328      1
261600010045      1
331700011524      1
342400010113      1
Name: count, Length: 2974, dtype: int64

#### Pivoting the exam results

In [45]:
df_y_t = df_y_t.pivot(index=['ENTITY_CD'], 
                            values=["PER_PROF"], 
                            columns=["SUBJECT"]
                            )

And we can see that the table pivots with no trouble, yay! 

In [51]:
df_y_t.head()

PER_PROF  \
SUBJECT     Regents Common Core Algebra I   
ENTITY_CD                                   
10100860907                            40   
10615020000                           100   
10615020001                           100   
10623060006                             s   
10623060007                             s   

                                                                                
SUBJECT     Regents Common Core English Language Art Regents Phy Set/Earth Sci  
ENTITY_CD                                                                       
10100860907                                       70                        00  
10615020000                                      NaN                       100  
10615020001                                      NaN                       100  
10623060006                                      NaN                       NaN  
10623060007                                      NaN                       NaN

### Merging the Tables

In [47]:
from functools import reduce

data_frames_to_merge = [df_epp_t, df_nxp_t, df_ocert_t, df_y_t]
def merge_function(left_df, right_df):
    print("Initializing")
    return pd.merge(left_df, right_df, on='ENTITY_CD',
                                            how='inner'
                                            )
df = reduce(merge_function, data_frames_to_merge)

Initializing
Initializing
Initializing


MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

The merge failed because the pivot table has rows/columns at the top that are wonky. We need to look into how to fix that. 